In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from rake_nltk import Rake
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

pd.set_option('display.max_colwidth', None)

# df = pd.read_excel("raw data/LG-A9K_REVIEWS.xlsx")

df = pd.read_csv("raw data/LG-A9K_REVIEWS.csv")
# df = pd.read_csv("raw data/LG-A9K_REVIEWS.csv", encoding='cp1252')
df.insert(0, 'ID', range(0 + len(df)))
df['REVIEW'] = df['TITLE'] + ". " + df['CONTENT']
df['REVIEW'] = df['REVIEW'].replace(np.nan, '')
df = df.drop(columns=['TITLE', 'CONTENT'])

print(df.head())

   ID RETAILER     PRODUCT  RATING POST_DATE REVIEWER_NAME  \
0   0  bestbuy  LG A925KSM       5  ########   Curmudgeon1   
1   1  bestbuy  LG A925KSM       5  ########    Pilgrim007   
2   2  bestbuy  LG A925KSM       5  ########      MrGadget   
3   3  bestbuy  LG A925KSM       5  ########      TechGeek   
4   4  bestbuy  LG A925KSM       5  ########       DeeGore   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [55]:
# Remove empty strings after colon and add space before Pros and Cons
df['REVIEW'] = df['REVIEW'].apply(lambda x: x.replace(":   ",":").replace(":  ",":").replace(": ",":").replace(":\n\n", ": ").replace(":\n", ": ").replace("\t","").replace("\n-","").replace("\n ","\n"))
# df['REVIEW'][0]


df['REVIEW']

0       POWERFUL, LIGHTWEIGHT & EFFICIENT STICK VACUUM. The LG CordZero A925KSM Kompressor Stick Vacuum is similar to some other LG cordless vacuum models, and that is a very good thing. The LG A925KSM is an excellent cordless stick vacuum that performs very well and is very pleasant and easy to use.\n\nThe LG CordZero A925KSM Kompressor arrives very well packed and includes the vacuum motor power head, an aluminum telescoping extension wand (20” collapsed, 29.5” extended), a universal power nozzle (for carpet and bare floors), a combination tool (with removable brush), a crevice tool, a free-standing / wall mountable, portable storage stand with integral battery charger, two lithium ion rechargeable batteries, a spare pre-filter for the motor and a small brush for cleaning the dirt collection canister and motor housing.\n\nAssembly of the charging stand is straightforward, and LG provides a quantity of printed product documentation. The stand has a weighted base, which I found to be v

In [56]:
# Separate REVIEW into PARAGRAPHS
def separate_paragraphs(review):
    para_list = []
    paragraphs = review.split('\n\n')
    para_list.extend(paragraphs)
    return para_list

df['PARAGRAPHS'] = df['REVIEW'].apply(lambda x: separate_paragraphs(x))

# Remove empty strings from PARAGRAPHS
# df['PARAGRAPHS'] = df['PARAGRAPHS'].apply(lambda x: list(filter(None, x)))
# df['PARAGRAPHS'][0]

df['PARAGRAPHS'][0]

['POWERFUL, LIGHTWEIGHT & EFFICIENT STICK VACUUM. The LG CordZero A925KSM Kompressor Stick Vacuum is similar to some other LG cordless vacuum models, and that is a very good thing. The LG A925KSM is an excellent cordless stick vacuum that performs very well and is very pleasant and easy to use.',
 'The LG CordZero A925KSM Kompressor arrives very well packed and includes the vacuum motor power head, an aluminum telescoping extension wand (20” collapsed, 29.5” extended), a universal power nozzle (for carpet and bare floors), a combination tool (with removable brush), a crevice tool, a free-standing / wall mountable, portable storage stand with integral battery charger, two lithium ion rechargeable batteries, a spare pre-filter for the motor and a small brush for cleaning the dirt collection canister and motor housing.',
 'Assembly of the charging stand is straightforward, and LG provides a quantity of printed product documentation. The stand has a weighted base, which I found to be very 

In [57]:
# Drop df columns: REVIEW
df = df.drop(columns=['RETAILER', 'PRODUCT','POST_DATE', 'REVIEWER_NAME','REVIEW'])

# Make PARAGRAPHS list to string
# df['PARAGRAPHS'] = df['PARAGRAPHS'].agg(lambda x: ','.join(map(str, x)))
df.head()

# df.insert(1, 'P_ID', range(0 + len(df)))





ID  RATING  \
0   0       5   
1   1       5   
2   2       5   
3   3       5   
4   4       5   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [58]:
# explode the PARAGRAPHS Column
df = df.explode('PARAGRAPHS')
df = df.reset_index(drop=True)
# remove empty rows from PARAGRAPHS
df_paragraphs = df[df['PARAGRAPHS'] != '']

df_paragraphs

,ID,RATING,PARAGRAPHS
0,0,5,"POWERFUL, LIGHTWEIGHT & EFFICIENT STICK VACUUM. The LG CordZero A925KSM Kompressor Stick Vacuum is similar to some other LG cordless vacuum models, and that is a very good thing. The LG A925KSM is an excellent cordless stick vacuum that performs very well and is very pleasant and easy to use."
1,0,5,"The LG CordZero A925KSM Kompressor arrives very well packed and includes the vacuum motor power head, an aluminum telescoping extension wand (20” collapsed, 29.5” extended), a universal power nozzle (for carpet and bare floors), a combination tool (with removable brush), a crevice tool, a free-standing / wall mountable, portable storage stand with integral battery charger, two lithium ion rechargeable batteries, a spare pre-filter for the motor and a small brush for cleaning the dirt collection canister and motor housing."
2,0,5,"Assembly of the charging stand is straightforward, and LG provides a quantity of printed product documentation. The stand has a weighted base, which I found to be very stable. The stand may also be wall mounted. The hardware for wall attachment is included. Notably, all of the included components and accessories are accommodated on the charging stand!"
3,0,5,"The combination tool and crevice tool attachments can be used with or without the telescoping extension wand, easily accommodating cleaning out of reach areas, as well as close-up cleaning tasks such as upholstered furniture. The telescoping wand provides four latching positions. All of the attachments easily snap together and latch securely."
4,0,5,"Cordless stick vacuums are particularly attractive and desirable for their light weight, easy maneuverability and superior convenience. The LG CordZero A925 Kompressor excels in those attributes."
...,...,...,...
2704,1225,5,"The most amazing thing!. I have to say I was blown away by this product. I didn't expect to love it so much, but LG went above and beyond to make this easy for the buyer, and effortless for cleaning. This is likely the best purchase I've made this year."
2705,1226,5,"Perfection. I am obsessed with this vacuum the two recharging batteries, the self cleaning on its on makes it worth it alone. Easy to use and great on all surfaces."
2706,1226,5,"Light weight, doesn’t require lots of space, this is the all time best vacuum I have ever had!!"
2707,1227,5,Great Stick Vaccum. I’m so in love with my LG Cord stiick vacuum with charging station.It has such a powerful suction.It’s light weight but so durable .It empty’s it’s self in a container .It has a charging station which is awesome.This is the best Vacuum i’ve ever had .It’s amazing .


In [59]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

from scipy.special import softmax



In [60]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [61]:
from tqdm.notebook import tqdm
def polarity_scores_roberta(review):
    encoded_text = tokenizer(review, padding=True, truncation=True, max_length=512,  return_tensors='pt' )
    encoded_text
    # print(encoded_text)
    output = model(**encoded_text)
    # output
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores
    scores_dict = {
        'negative' : scores[0],
        'neutral' :  scores[1],
        'positive' : scores[2]
    }
    return scores_dict
# polarity_scores_roberta(review)

In [62]:
df_paragraphs.insert(1, 'P_ID', range(0 + len(df_paragraphs)))

df_paragraphs


,ID,P_ID,RATING,PARAGRAPHS
0,0,0,5,"POWERFUL, LIGHTWEIGHT & EFFICIENT STICK VACUUM. The LG CordZero A925KSM Kompressor Stick Vacuum is similar to some other LG cordless vacuum models, and that is a very good thing. The LG A925KSM is an excellent cordless stick vacuum that performs very well and is very pleasant and easy to use."
1,0,1,5,"The LG CordZero A925KSM Kompressor arrives very well packed and includes the vacuum motor power head, an aluminum telescoping extension wand (20” collapsed, 29.5” extended), a universal power nozzle (for carpet and bare floors), a combination tool (with removable brush), a crevice tool, a free-standing / wall mountable, portable storage stand with integral battery charger, two lithium ion rechargeable batteries, a spare pre-filter for the motor and a small brush for cleaning the dirt collection canister and motor housing."
2,0,2,5,"Assembly of the charging stand is straightforward, and LG provides a quantity of printed product documentation. The stand has a weighted base, which I found to be very stable. The stand may also be wall mounted. The hardware for wall attachment is included. Notably, all of the included components and accessories are accommodated on the charging stand!"
3,0,3,5,"The combination tool and crevice tool attachments can be used with or without the telescoping extension wand, easily accommodating cleaning out of reach areas, as well as close-up cleaning tasks such as upholstered furniture. The telescoping wand provides four latching positions. All of the attachments easily snap together and latch securely."
4,0,4,5,"Cordless stick vacuums are particularly attractive and desirable for their light weight, easy maneuverability and superior convenience. The LG CordZero A925 Kompressor excels in those attributes."
...,...,...,...,...
2704,1225,2373,5,"The most amazing thing!. I have to say I was blown away by this product. I didn't expect to love it so much, but LG went above and beyond to make this easy for the buyer, and effortless for cleaning. This is likely the best purchase I've made this year."
2705,1226,2374,5,"Perfection. I am obsessed with this vacuum the two recharging batteries, the self cleaning on its on makes it worth it alone. Easy to use and great on all surfaces."
2706,1226,2375,5,"Light weight, doesn’t require lots of space, this is the all time best vacuum I have ever had!!"
2707,1227,2376,5,Great Stick Vaccum. I’m so in love with my LG Cord stiick vacuum with charging station.It has such a powerful suction.It’s light weight but so durable .It empty’s it’s self in a container .It has a charging station which is awesome.This is the best Vacuum i’ve ever had .It’s amazing .


In [63]:
res = {}
for i, row in tqdm(df_paragraphs.iterrows(), total=len(df_paragraphs)):
    try:
        text = row['PARAGRAPHS']
        myid = row['P_ID']
        # vader_results = sia.polarity_scores(text)
        roberta_result = polarity_scores_roberta(text)
        res[myid] = {**roberta_result}
    except RuntimeError:
        print(f'Broke for id {myid}')

results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'P_ID'})
results_df = results_df.merge(df_paragraphs, how='left')
results_df.head()

  0%|          | 0/2378 [00:00<?, ?it/s]

,P_ID,negative,neutral,positive,ID,RATING,PARAGRAPHS
0,0,0.001277,0.012046,0.986677,0,5,"POWERFUL, LIGHTWEIGHT & EFFICIENT STICK VACUUM. The LG CordZero A925KSM Kompressor Stick Vacuum is similar to some other LG cordless vacuum models, and that is a very good thing. The LG A925KSM is an excellent cordless stick vacuum that performs very well and is very pleasant and easy to use."
1,1,0.002491,0.235200,0.762309,0,5,"The LG CordZero A925KSM Kompressor arrives very well packed and includes the vacuum motor power head, an aluminum telescoping extension wand (20” collapsed, 29.5” extended), a universal power nozzle (for carpet and bare floors), a combination tool (with removable brush), a crevice tool, a free-standing / wall mountable, portable storage stand with integral battery charger, two lithium ion rechargeable batteries, a spare pre-filter for the motor and a small brush for cleaning the dirt collection canister and motor housing."
2,2,0.003077,0.128773,0.868150,0,5,"Assembly of the charging stand is straightforward, and LG provides a quantity of printed product documentation. The stand has a weighted base, which I found to be very stable. The stand may also be wall mounted. The hardware for wall attachment is included. Notably, all of the included components and accessories are accommodated on the charging stand!"
3,3,0.007611,0.393929,0.598460,0,5,"The combination tool and crevice tool attachments can be used with or without the telescoping extension wand, easily accommodating cleaning out of reach areas, as well as close-up cleaning tasks such as upholstered furniture. The telescoping wand provides four latching positions. All of the attachments easily snap together and latch securely."
4,4,0.001160,0.025889,0.972951,0,5,"Cordless stick vacuums are particularly attractive and desirable for their light weight, easy maneuverability and superior convenience. The LG CordZero A925 Kompressor excels in those attributes."


In [64]:
# Keywords Extraction
# added below since giving a certificate verification error.
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aiden2.kim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aiden2.kim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [93]:
# copy results_df to keywords_df
keywords_df = results_df.copy()
keywords_df['PARAGRAPHS'][524]

'And yes. This is a lot better than v11 dyson.\nFeels premium'

In [94]:
import string, re
from nltk import word_tokenize
from nltk.corpus import stopwords

for i in range(0, len(keywords_df['PARAGRAPHS'])):
    keywords_df['PARAGRAPHS'][i] = keywords_df['PARAGRAPHS'][i].translate(str.maketrans('', '', string.punctuation))
    keywords_df['PARAGRAPHS'][i] = keywords_df['PARAGRAPHS'][i].replace('\n', '. ')
    keywords_df['PARAGRAPHS'][i] = keywords_df['PARAGRAPHS'][i].lower()
    # keywords_df['PARAGRAPHS'][i] = re.sub("['\"]","",keywords_df['PARAGRAPHS'][i])
    for j in re.findall('"([^"]*)"', keywords_df['PARAGRAPHS'][i]):
            keywords_df['PARAGRAPHS'][i] = keywords_df['PARAGRAPHS'][i].replace('"{}"'.format(j), j.replace(' ', '_'))



keywords_df['PARAGRAPHS'][524]


C:\Users\aiden2.kim\AppData\Local\Temp\ipykernel_1436\3065378764.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keywords_df['PARAGRAPHS'][i] = keywords_df['PARAGRAPHS'][i].translate(str.maketrans('', '', string.punctuation))
C:\Users\aiden2.kim\AppData\Local\Temp\ipykernel_1436\3065378764.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keywords_df['PARAGRAPHS'][i] = keywords_df['PARAGRAPHS'][i].replace('\n', '. ')
C:\Users\aiden2.kim\AppData\Local\Temp\ipykernel_1436\3065378764.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

'and yes this is a lot better than v11 dyson. feels premium'

In [95]:
# word_tokenize paragraphs

keywords_df['KEYWORD'] = keywords_df['PARAGRAPHS'].apply(lambda x: word_tokenize(x))
keywords_df['KEYWORD']

0                                                                                                                                                                                                                                                                    [powerful, lightweight, efficient, stick, vacuum, the, lg, cordzero, a925ksm, kompressor, stick, vacuum, is, similar, to, some, other, lg, cordless, vacuum, models, and, that, is, a, very, good, thing, the, lg, a925ksm, is, an, excellent, cordless, stick, vacuum, that, performs, very, well, and, is, very, pleasant, and, easy, to, use]
1       [the, lg, cordzero, a925ksm, kompressor, arrives, very, well, packed, and, includes, the, vacuum, motor, power, head, an, aluminum, telescoping, extension, wand, 20, ”, collapsed, 295, ”, extended, a, universal, power, nozzle, for, carpet, and, bare, floors, a, combination, tool, with, removable, brush, a, crevice, tool, a, freestanding, wall, mountable, portable, storage, stand, with, integ

In [96]:
english_stopwords = stopwords.words('english')
for i in range(0, len(keywords_df['KEYWORD'])):
    keywords_df['KEYWORD'][i] = [w for w in keywords_df['KEYWORD'][i] if w.lower() not in english_stopwords]



# remove duplicate keywords_df['KEYWORD']
keywords_df['KEYWORD'] = keywords_df['KEYWORD'].apply(lambda x: list(dict.fromkeys(x)))
keywords_df['KEYWORD']

C:\Users\aiden2.kim\AppData\Local\Temp\ipykernel_1436\2784093129.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keywords_df['KEYWORD'][i] = [w for w in keywords_df['KEYWORD'][i] if w.lower() not in english_stopwords]


0                                                                                                                                                                                                                                                                                                    [powerful, lightweight, efficient, stick, vacuum, lg, cordzero, a925ksm, kompressor, similar, cordless, models, good, thing, excellent, performs, well, pleasant, easy, use]
1       [lg, cordzero, a925ksm, kompressor, arrives, well, packed, includes, vacuum, motor, power, head, aluminum, telescoping, extension, wand, 20, ”, collapsed, 295, extended, universal, nozzle, carpet, bare, floors, combination, tool, removable, brush, crevice, freestanding, wall, mountable, portable, storage, stand, integral, battery, charger, two, lithium, ion, rechargeable, batteries, spare, prefilter, small, cleaning, dirt, collection, canister, housing]
2                                                                   

In [97]:
# apply rake_nltk_var.extract_keywords_from_text to each row in PARAGRAPHS
# keywords_df['KEYWORDS'] = keywords_df['PARAGRAPHS'].apply(lambda x: rake_nltk_var.extract_keywords_from_text(x))
# keywords_df['KEYWORDS'] = keywords_df['KEYWORDS'].apply(lambda x: rake_nltk_var.get_ranked_phrases())
# keywords_df

In [98]:
keywords_df['PARAGRAPHS'][524]

'and yes this is a lot better than v11 dyson. feels premium'

In [99]:
from rake_nltk import Metric
r = Rake(include_repeated_phrases=False,
         min_length=2,
         ranking_metric=Metric.WORD_DEGREE)

In [100]:
keywords = []
for i in range(0, len(keywords_df['PARAGRAPHS'])):
    keyword = r.extract_keywords_from_text(keywords_df['PARAGRAPHS'][i])
    keyword = r.get_ranked_phrases()
    keywords.append(keyword)
keywords_df['KEYWORDS'] = keywords

In [101]:
len(keywords)

2378

In [102]:
keywords_df['KEYWORD'][524]

['yes', 'lot', 'better', 'v11', 'dyson', '.', 'feels', 'premium']

In [103]:
keywords_df['KEYWORDS'][524]

['v11 dyson', 'lot better', 'feels premium']

In [75]:
# save to csv
keywords_df.to_csv('keywords_df_RAKE.csv', encoding='utf-8-sig',index=False)


PermissionError: [Errno 13] Permission denied: 'keywords_df_newline.csv'

In [107]:
from pke.unsupervised import YAKE
from nltk.corpus import stopwords

document = "Machine learning (ML) is the study of computer algorithms that improve automatically through experience. It is seen as a subset of artificial intelligence."

# 1. Create YAKE keyword extractor
extractor = YAKE()

# 2. Load document
extractor.load_document(input=document,
                        language='en',
                        normalization=None)


# 3. Generate candidate 1-gram and 2-gram keywords
stoplist = stopwords.words('english')
extractor.candidate_selection(n=2, stoplist=stoplist)


# 4. Calculate scores for the candidate keywords
extractor.candidate_weighting(window=2,
                              stoplist=stoplist,
                              use_stems=False)

# 5. Select 10 highest ranked keywords
# Remove redundant keywords with similarity above 80%
key_phrases = extractor.get_n_best(n=10, threshold=0.8)
print(key_phrases)

TypeError: YAKE.candidate_selection() got an unexpected keyword argument 'stoplist'

In [109]:
import yake
document = "Machine learning (ML) is the study of computer algorithms that improve automatically through experience. It is seen as a subset of artificial intelligence."
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(document)

for kw in keywords:
	print(kw)

('Machine learning', 0.01552184797949213)
('automatically through experience', 0.02541909697430123)
('study of computer', 0.04188746641162499)
('computer algorithms', 0.04188746641162499)
('algorithms that improve', 0.04188746641162499)
('improve automatically', 0.04188746641162499)
('Machine', 0.12363091320521931)
('learning', 0.12363091320521931)
('experience', 0.12363091320521931)
('artificial intelligence', 0.18075564686791562)
('study', 0.2005079697193566)
('computer', 0.2005079697193566)
('algorithms', 0.2005079697193566)
('improve', 0.2005079697193566)
('automatically', 0.2005079697193566)
('subset of artificial', 0.2766850298235749)
('intelligence', 0.32883716426354986)
('subset', 0.46553351027698087)
('artificial', 0.46553351027698087)


In [110]:
language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)

for kw in keywords:
    print(kw)

('Wow', 0.10011447124450398)
('quiet and powerful', 0.1933129828927864)
('length is huge', 0.2618459888686481)
('adjustable length', 0.32508082564191815)
('kids', 0.33105685691927955)
('dog', 0.33105685691927955)
('charging dock', 0.3443387535828273)
('Game changer', 0.3822919834808006)
('quiet', 0.40248836299570595)
('powerful', 0.40248836299570595)
('household', 0.40248836299570595)
('huge', 0.41895278496570726)
('vacuum empties', 0.4247678766088889)
('dock', 0.46910861582176344)
('adjustable', 0.495307030195649)
('length', 0.495307030195649)
('comfortable', 0.495307030195649)
('house', 0.495307030195649)
('Game', 0.5258933763376524)
('changer', 0.5258933763376524)
